In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import warnings
import itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Company ticker
company = "MSFT"

# Initialize dictionary to store results
results = []

# Create a Ticker object for the company
ticker = yf.Ticker(company)

# Fetch weekly historical market data for the last 3 years
hist = ticker.history(period="3y", interval="1wk")

# Extract the 'Close' prices and convert to a DataFrame
data = hist['Close'].reset_index()
data.columns = ['Date', 'Close']

# Set 'Date' as the index
data.set_index('Date', inplace=True)

# Handle missing values by forward fill
data.fillna(method='ffill', inplace=True)

# Resample to weekly data and use mean as aggregation
data_resampled = data.resample('W').mean()

# Split data into training and testing sets
train_size = int(0.8 * len(data_resampled))
train_data, test_data = data_resampled.iloc[:train_size], data_resampled.iloc[train_size:]

# Define SARIMA order and seasonal order parameters to try
p_values = range(0, 3)
d_values = range(0, 2)
q_values = range(0, 3)
P_values = range(0, 3)
D_values = range(0, 2)
Q_values = range(0, 3)
s = 52  # Seasonal period

# Generate all possible combinations of SARIMA orders and seasonal orders
order_combinations = list(itertools.product(p_values, d_values, q_values))
seasonal_order_combinations = list(itertools.product(P_values, D_values, Q_values, [s]))

# Define variations of trend options
trend_options = ['n', 'c', 't', 'ct']

for order in order_combinations:
    for seasonal_order in seasonal_order_combinations:
        for trend in trend_options:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                try:
                    model = SARIMAX(train_data, order=order, seasonal_order=seasonal_order, trend=trend)

                    fitted_model = model.fit()

                    # Forecast
                    forecast = fitted_model.forecast(steps=len(test_data))

                    # Calculate metrics
                    mpe = np.mean((test_data['Close'] - forecast) / test_data['Close']) * 100
                    wmpe = np.sum((test_data['Close'] - forecast) / test_data['Close'] * test_data['Close']) / np.sum(test_data['Close']) * 100
                    mape = np.mean(np.abs((test_data['Close'] - forecast) / test_data['Close'])) * 100
                    wmape = np.sum(np.abs(test_data['Close'] - forecast) / test_data['Close'] * test_data['Close']) / np.sum(test_data['Close']) * 100

                    # Store results in a list
                    results.append((order, seasonal_order, trend, mpe, wmpe, mape, wmape))

                except Exception as e:
                    print(f"An error occurred for Trend: {trend}, Order: {order}, Seasonal Order: {seasonal_order}: {e}")

# Create a DataFrame from the results
results_df = pd.DataFrame(results, columns=['Order', 'Seasonal Order', 'Trend', 'MPE', 'WMPE', 'MAPE', 'WMAPE'])

# Save the results to a CSV file
results_df.to_csv(f'{company}_sarimax_results.csv', index=False)

print(f"Results saved to '{company}_sarimax_results.csv'")
